**when sampling from k-dpp**

We shall sample until one of the following is met:

*  Number of samples is complete based on PCA on kernel (choice of k)
*  Stopping criterion based on wilcoxon non-parametric test (early stopping). Using library: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html

For speed we will demo the Nystroem kernel

Based on the following notebook: https://github.com/chappers/Context-driven-constraints-for-gradient-boosted-models/blob/master/autoML/streaming/dpp-groupfs.ipynb

In [1]:
import sklearn

In [2]:
from sklearn.datasets import make_regression, make_classification
from sklearn.linear_model import SGDRegressor

import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import wilcoxon
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from dpp import sample_dpp, decompose_kernel, sample_conditional_dpp

In [31]:
wilcoxon(np.random.normal(size=(100,)), np.random.normal(size=(100,))).pvalue

0.75436608850453835

In [3]:
X, y = make_regression()
pdf = pd.DataFrame(X)
pdf.columns = ['c{}'.format(x) for x in range(100)]

In [4]:
X.shape

(100, 100)

In [5]:
X1 = pdf[['c{}'.format(x) for x in range(50, 100)]]
X2 = pdf[['c{}'.format(x) for x in range(50)]]

In [6]:
[idx for idx, x in enumerate(pdf.columns) if x in ['c0', 'c13']]

[0, 13]

In [34]:
wilcoxon([1,2,3,4,5], [3,4,5,6,1])

c:\users\chapm\anaconda3\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


WilcoxonResult(statistic=5.0, pvalue=0.47950012218695348)

In [48]:
"""
Implement DPP version that is similar to what is done above


sketch of solution
------------------

DPP requires a known number of parameters to check at each partial fit!


"""

class DPPRegressor(SGDRegressor):
    def __init__(self, loss="squared_loss", penalty="l2", alpha=0.0001,
                 l1_ratio=0.15, fit_intercept=True, max_iter=None, tol=None,
                 shuffle=True, verbose=0, epsilon=0.1,
                 random_state=None, learning_rate="invscaling", eta0=0.01,
                 power_t=0.25, warm_start=False, average=False, n_iter=None,
                 intragroup_alpha=0.05, intergroup_thres=None):
        super(DPPRegressor, self).__init__(loss=loss, penalty=penalty,
                                           alpha=alpha, l1_ratio=l1_ratio,
                                           fit_intercept=fit_intercept,
                                           max_iter=max_iter, tol=tol,
                                           shuffle=shuffle,
                                           verbose=verbose,
                                           epsilon=epsilon,
                                           random_state=random_state,
                                           learning_rate=learning_rate,
                                           eta0=eta0, power_t=power_t,
                                           warm_start=warm_start,
                                           average=average, n_iter=n_iter)
        self.coef_info = {'cols': [], 'coef':[], 'excluded_cols': []}
        self.seen_cols = []
        self.base_shape = None
        self.intragroup_alpha = intragroup_alpha
        self.intergroup_thres = intergroup_thres if intergroup_thres is not None else epsilon
    
    def _dpp_estimate_k(self, L):
        """
        L is the input kernel
        """
        pca = PCA(n_components=None)
        pca.fit(L)
        return np.min(np.argwhere(np.cumsum(pca.explained_variance_ratio_) > 
                                  (1-self.intragroup_alpha)))
        
    
    def add_column_exclusion(self, cols):
        self.coef_info['excluded_cols'] = list(self.coef_info['excluded_cols']) + list(cols)
        
    def _fit_columns(self, X_, return_x=True, transform_only=False):
        """
        Method filter through "unselected" columns. The goal of this 
        method is to filter any uninformative columns.
        
        This will be selected based on index only?
        
        If return_x is false, it will only return the boolean mask.
        """
        X = X_[X_.columns.difference(self.coef_info['excluded_cols'])]
        
        # order the columns correctly...
        col_order = self.coef_info['cols'] + list([x for x in X.columns if x not in self.coef_info['cols']])
        X = X[col_order]
        return X

    def _reg_penalty(self, X):
        col_coef = [(col, coef) for col, coef in zip(X.columns.tolist(), self.coef_) if np.abs(coef) >= self.intergroup_thres]
        self.coef_info['cols'] = [x for x, _ in col_coef]
        self.coef_info['coef'] = [x for _, x in col_coef]
        self.coef_info['excluded_cols'] = [x for x in self.seen_cols if x not in self.coef_info['cols']]
        self.coef_ = np.array(self.coef_info['coef'])  
    
    def _dpp_sel(self, X_, y=None):
        """
        DPP only relies on X. 
        
        We will condition the sampling based on:
        *  `self.coef_info['cols']`
        """
        X = np.array(X_)
        if X.shape[0] < 1000:
            feat_dist = rbf_kernel(X.T)
        else:
            feat_dist = Nystroem().fit_transform(X.T)
        k = self._dpp_estimate_k(feat_dist) - len(self.coef_info['cols'])
                
        if len(self.coef_info['cols']) == 0:
            feat_index = sample_dpp(decompose_kernel(feat_dist), k=k)
        else:
            cols_to_index = [idx for idx, x in enumerate(X_.columns) if x in self.coef_info['cols']]
            feat_index = sample_conditional_dpp(feat_dist, cols_to_index, k=k)
        
        # iterate over feat_index to determine 
        # information on wilcoxon test
        feat_check = []
        excl_check = []
        for idx1, val1 in enumerate(feat_index):
            for idx2, val2 in enumerate(feat_index):
                if idx1 == 0:
                    feat_check.append(val1)
                if idx1 <= idx2 or idx1 in excl_check or idx2 in excl_check:
                    continue
                
                if wilcoxon(X[:, val1], X[:, val2]).pvalue >= self.intragroup_alpha:
                    feat_check.append(val1)
                    feat_check.append(val2)
                    feat_check = list(set(feat_check))
                else:
                    excl_check.append(val2)
        
        index_to_col = [col for idx, col in enumerate(X_.columns) if idx in feat_check]
        self.coef_info['cols'] = list(set(self.coef_info['cols'] + index_to_col))
        col_rem = X_.columns.difference(self.coef_info['cols'])
        self.add_column_exclusion(col_rem)        
        
    def fit(self, X, y, coef_init=None, intercept_init=None,
            sample_weight=None):
        self.seen_cols = list(set(self.seen_cols + X.columns.tolist()))
        
        # TODO: add DPP selection
        self.coef_info = {'cols': [], 'coef':[], 'excluded_cols': []}
        self._dpp_sel(X, y)
        X = self._fit_columns(X)
        
        super(DPPRegressor, self).fit(X, y, coef_init=coef_init, intercept_init=intercept_init,
            sample_weight=sample_weight)
        self._reg_penalty(X)
        return self
    
    def partial_fit(self, X, y, sample_weight=None):
        X_ = X.copy()
        self.seen_cols = list(set(self.seen_cols + X.columns.tolist()))
        X = X[X.columns.difference(self.coef_info['excluded_cols'])]
        
        # TODO: add DPP selection
        self._dpp_sel(X, y)
        X = self._fit_columns(X_)
        
        # now update coefficients
        n_samples, n_features = X.shape
        coef_list = np.zeros(n_features, dtype=np.float64, order="C")
        coef_list[:len(self.coef_info['coef'])] = self.coef_info['coef']
        self.coef_ = coef_list.copy()
        
        super(DPPRegressor, self).partial_fit(X, y, sample_weight=None)  
        self._reg_penalty(X)
        return self
    
    def predict(self, X):
        X = self._fit_columns(X, transform_only=True)
        return super(DPPRegressor, self).predict(X)        

In [49]:
model = DPPRegressor(max_iter=1000)
model.fit(X1, y)

DPPRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, intergroup_thres=0.1, intragroup_alpha=0.05,
       l1_ratio=0.15, learning_rate='invscaling', loss='squared_loss',
       max_iter=1000, n_iter=None, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [50]:
len(model.coef_)

40

In [51]:
model.partial_fit(pdf, y)

c:\users\chapm\anaconda3\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
c:\users\chapm\anaconda3\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
c:\users\chapm\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\users\chapm\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\users\chapm\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


DPPRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, intergroup_thres=0.1, intragroup_alpha=0.05,
       l1_ratio=0.15, learning_rate='invscaling', loss='squared_loss',
       max_iter=1000, n_iter=None, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [46]:
len(model.coef_)

56

In [47]:
model.predict(pdf)

array([-212.94777128,  320.21082479,  -96.46891294,  -54.78595823,
        -36.81428094,   54.87168851,  -93.49542057,  103.95038936,
        165.26515227,  -74.73007888, -118.87163358,   96.5536367 ,
          5.24539943,  238.41595736,  -29.50419503,   43.55234033,
        -30.19621481,    3.04936598,  -16.29435966,  160.92544535,
       -222.25043937,  444.40908113,  -22.92154964,  -67.71627381,
       -200.55852412,  423.52242524,  173.16823766,   55.64052251,
        108.5712764 ,  -72.14247769, -211.28739099, -172.04297086,
       -183.43826894,  302.85898519,  401.76914218,    1.4540655 ,
       -309.44659618,    4.71206652,  -96.63542729,   13.33204371,
       -437.02168181,  211.83758874,  383.41515714,   37.40018305,
         69.25207111,  133.20777767,  -44.94898174, -365.76751482,
        222.29835922,   47.15305935, -102.24462188, -288.69831597,
       -338.95099292, -456.38420546, -224.00298753,  -47.00136985,
        186.24862704, -201.76658292, -307.67925744, -168.16064